In [1]:
import torch

In [36]:
q_len = 8
k_len = 8

mask = torch.triu(torch.ones((q_len, k_len), dtype=int), diagonal=1)

print(mask)

position_ids = torch.tensor([[0, 1, 2, 3, 4, 0, 1, 2]]).int()

print(position_ids)

print((position_ids.unsqueeze(-1) >= position_ids.unsqueeze(-2)))

# find all indices where position_ids == 0
starts = (position_ids == 0).to(torch.long)
segment_ids = starts.cumsum(dim=1)

print(segment_ids)

mask = mask | (segment_ids.unsqueeze(-1) != segment_ids.unsqueeze(-2))

print(mask)

tensor([[0, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[0, 1, 2, 3, 4, 0, 1, 2]], dtype=torch.int32)
tensor([[[ True, False, False, False, False,  True, False, False],
         [ True,  True, False, False, False,  True,  True, False],
         [ True,  True,  True, False, False,  True,  True,  True],
         [ True,  True,  True,  True, False,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True],
         [ True, False, False, False, False,  True, False, False],
         [ True,  True, False, False, False,  True,  True, False],
         [ True,  True,  True, False, False,  True,  True,  True]]])
tensor([[1, 1, 1, 1, 1, 2, 2, 2]])
tensor([[[0, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 1, 1, 1, 1,

In [39]:
input_ids = torch.tensor([
    [0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0],
])
eos_token_id = 1
pad_token_id = 2

def make_position_ids(input_ids: torch.Tensor, eos_token_id: int, pad_token_id: int | None = None):
    """
    input_ids: (B, S) LongTensor
    Reset rule: positions start at 0; after every EOS token, the *next* token restarts at 0.
    pad handling: if pad_token_id is provided, pad positions are set to 0.
    """
    B, S = input_ids.shape
    device = input_ids.device

    # segment starts: col 0, and any position whose *previous* token was EOS
    starts = torch.zeros_like(input_ids, dtype=torch.bool)
    starts[:, 0] = True
    starts[:, 1:] = (input_ids[:, :-1] == eos_token_id)

    # index grid
    idx = torch.arange(S, device=device).unsqueeze(0).expand(B, -1)

    # index of the most recent segment start, per position (running max trick)
    last_start_idx = torch.where(starts, idx, torch.full_like(idx, -1))
    print(last_start_idx)
    last_start_idx = torch.cummax(last_start_idx, dim=1).values  # (B, S)
    print(last_start_idx)

    # distance since last start → 0,1,2,... within each segment
    position_ids = (idx - last_start_idx).to(torch.long)

    # optional: zero-out pads
    if pad_token_id is not None:
        position_ids = position_ids.masked_fill(input_ids == pad_token_id, 0)

    return position_ids

print(make_position_ids(input_ids, eos_token_id, pad_token_id))

tensor([[ 0, -1, -1, -1, -1,  5, -1, -1],
        [ 0, -1, -1, -1, -1, -1,  6, -1]])
tensor([[0, 0, 0, 0, 0, 5, 5, 5],
        [0, 0, 0, 0, 0, 0, 6, 6]])
tensor([[0, 1, 2, 3, 4, 0, 1, 2],
        [0, 1, 2, 3, 4, 5, 0, 1]])
